In [2]:
# Cell 1: Setup (Imports, Paths, Constants) - Updated OUTPUT_DIR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
import os
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

print("--- Running Setup ---")
# Add project root to Python path (assuming this notebook is in Project/notebooks/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to sys.path")

# Import necessary functions from src directory (based on your notebooks)
try:
    from src.io_utils import load_face_data
    from src.embedding import run_pca, run_mds, run_isomap, run_lle, run_tsne, run_spectral
    from src.metrics import compute_all_metrics
    print("Successfully imported functions from src.")
except ImportError as e:
    print(f"Failed to import from src: {e}. Ensure src/* exists.")
    load_face_data, run_pca, run_mds, run_isomap, run_lle, run_tsne, run_spectral, compute_all_metrics = (None,) * 8

# --- Constants and Parameters ---
OUTPUT_DIR = "../figs/" # <--- MODIFIED HERE
DATA_DIR = "../data/" # Relative to notebooks directory
FACE_FILE = "face.mat"
N_COMPONENTS = 2
RANDOM_STATE = 42
METRICS_K_FOR_TC = 5

# Parameter ranges (use the refined ones from 04_parameter_tuning)
k_values_refined = [3, 4, 5, 6, 7, 8, 9, 10, 12]
perplexity_values_refined = [5, 8, 10, 13, 15, 18, 20, 25, 30]

# Optimal parameters found in 04_parameter_tuning
optimal_params = {
    'PCA': {'n_components': N_COMPONENTS},
    'MDS': {'n_components': N_COMPONENTS, 'random_state': RANDOM_STATE, 'n_init': 4, 'max_iter': 300, 'n_jobs': -1}, # Updated based on embedding.py
    'Isomap': {'n_components': N_COMPONENTS, 'n_neighbors': 5},
    'LLE': {'n_components': N_COMPONENTS, 'n_neighbors': 5, 'method': 'standard', 'random_state': RANDOM_STATE},
    'LTSA': {'n_components': N_COMPONENTS, 'n_neighbors': 10, 'method': 'ltsa', 'random_state': RANDOM_STATE},
    'MLLE': {'n_components': N_COMPONENTS, 'n_neighbors': 12, 'method': 'modified', 'random_state': RANDOM_STATE},
    'HLLE': {'n_components': N_COMPONENTS, 'n_neighbors': 10, 'method': 'hessian', 'random_state': RANDOM_STATE},
    'Spectral': {'n_components': N_COMPONENTS, 'n_neighbors': 5, 'random_state': RANDOM_STATE},
    't-SNE': {'n_components': N_COMPONENTS, 'perplexity': 20, 'random_state': RANDOM_STATE}
}

# Functions mapping (adjust if your function names differ)
func_map = {
    'PCA': run_pca, 'MDS': run_mds, 'Isomap': run_isomap,
    'LLE': run_lle, 'LTSA': run_lle, 'MLLE': run_lle, 'HLLE': run_lle,
    'Spectral': run_spectral, 't-SNE': run_tsne
}

# Ensure output directory exists
output_dir_abs = os.path.abspath(os.path.join(os.getcwd(), OUTPUT_DIR)) # Use absolute path for check/create
if not os.path.exists(output_dir_abs):
    os.makedirs(output_dir_abs)
    print(f"Created output directory: {output_dir_abs}")
else:
    print(f"Output directory exists: {output_dir_abs}")


print("--- Setup Complete ---")

# Cell 2: Load and Prepare Data
print("\n--- Loading and Preparing Data ---")
face_data = None
flattened_face_data = None
ground_truth_order_indices = None

if load_face_data:
    # Adjust data_dir path to be relative to io_utils.py (which is in src/)
    face_data = load_face_data(data_dir='../data', file_name=FACE_FILE)
    if face_data is not None and face_data.ndim == 3 and face_data.shape[0] == 33:
         print(f"Data loaded successfully! Shape: {face_data.shape}")
         n_samples = face_data.shape[0]
         n_features = face_data.shape[1] * face_data.shape[2]
         flattened_face_data = face_data.reshape((n_samples, n_features))
         print(f"Data reshaped successfully to: {flattened_face_data.shape}")
         final_order = np.array([9, 20, 22, 14, 10, 4, 7, 0, 32, 13, 19, 12, 1, 30, 26, 11, 8, 5, 2, 29, 15, 16, 25, 21, 31, 17, 23, 6, 24, 27, 28, 3, 18])
         if len(final_order) == 33 and len(np.unique(final_order)) == 33:
             ground_truth_order_indices = final_order
             print("Ground Truth order defined.")
         else: print("Error: Ground Truth definition incorrect.")
    else: print("Error: Data loaded but shape is not the expected 3D shape.")
else: print("Error: load_face_data function not available.")

if flattened_face_data is not None:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(flattened_face_data)
    print("Data Scaled using StandardScaler.")
else:
    X_scaled = None
    print("Error: Cannot scale data as flattened_face_data is missing.")

print("--- Data Prep Complete ---")


# Cell 3: Generate Sample Faces Plot (Figure 1 in report)
print("\n--- Generating Sample Faces Plot ---")
if face_data is not None:
    fig_sample, axes_sample = plt.subplots(1, 3, figsize=(8, 3))
    fig_sample.suptitle('Sample Face Images')
    indices_to_show = [0, 16, 32]
    for i, img_index in enumerate(indices_to_show):
         img_matrix = face_data[img_index, :, :]
         axes_sample[i].imshow(img_matrix.T, cmap='gray')
         axes_sample[i].set_title(f'Index: {img_index}')
         axes_sample[i].axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    # Use absolute path for saving
    fig_filename = os.path.join(output_dir_abs, "sample_faces.png")
    plt.savefig(fig_filename)
    print(f"Saved: {fig_filename}")
    plt.close(fig_sample)
else:
    print("Skipping sample faces plot: face_data not loaded.")


# Cell 4: Compute Optimal Embeddings (Needed for Figures 2 & 3)
print("\n--- Computing Optimal Embeddings ---")
optimal_embeddings_dict = {}
if X_scaled is not None:
    start_optimal_time = time.time()
    for name, params in optimal_params.items():
        func = func_map.get(name)
        if func:
            print(f"Running {name} with optimal params...")
            try:
                 # Pass scaled data to embedding functions
                 optimal_embeddings_dict[name], _ = func(X_scaled, **params)
            except Exception as e:
                 print(f"{name} failed: {e}")
                 optimal_embeddings_dict[name] = None
        else: print(f"Function for {name} not found."); optimal_embeddings_dict[name] = None
    end_optimal_time = time.time()
    print(f"--- Optimal embeddings computed in {end_optimal_time - start_optimal_time:.2f} seconds ---")
else:
    print("Skipping optimal embeddings: Scaled data not available.")


# Cell 5: Generate Final Scatter Plot Grid (Figure 2 in report)
print("\n--- Generating Final Scatter Plot Grid ---")
if optimal_embeddings_dict:
    valid_optimal_embeddings = {name: emb for name, emb in optimal_embeddings_dict.items() if emb is not None}
    if valid_optimal_embeddings:
        n_methods_final = len(valid_optimal_embeddings)
        n_cols_final = 3
        n_rows_final = (n_methods_final + n_cols_final - 1) // n_cols_final
        fig_scatter, axes_scatter = plt.subplots(n_rows_final, n_cols_final, figsize=(n_cols_final * 4, n_rows_final * 3.5), squeeze=False)
        fig_scatter.suptitle('Final Embeddings (Optimal Parameters)', fontsize=14)
        axes_flat = axes_scatter.flatten()
        plot_idx = 0
        for name in sorted(valid_optimal_embeddings.keys()):
            emb = valid_optimal_embeddings[name]
            ax = axes_flat[plot_idx]
            scatter = ax.scatter(emb[:, 0], emb[:, 1], c=np.arange(emb.shape[0]), cmap='viridis', s=30)
            ax.set_title(f"{name}", fontsize=10)
            ax.set_xticks([]); ax.set_yticks([])
            plot_idx += 1
        for j in range(plot_idx, len(axes_flat)): axes_flat[j].axis('off')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig_filename = os.path.join(output_dir_abs, "final_scatter.png") # Use absolute path
        plt.savefig(fig_filename)
        print(f"Saved: {fig_filename}")
        plt.close(fig_scatter)
    else: print("No valid optimal embeddings to plot.")
else: print("Skipping scatter plot: optimal_embeddings_dict is empty.")


# Cell 6: Generate Final Image Plot Grid (Figure 3 in report)
print("\n--- Generating Final Image Plot Grid ---")
if optimal_embeddings_dict and face_data is not None:
    valid_optimal_embeddings_img = {name: emb for name, emb in optimal_embeddings_dict.items() if emb is not None}
    if valid_optimal_embeddings_img:
        n_methods_img = len(valid_optimal_embeddings_img)
        n_cols_img = 3
        n_rows_img = (n_methods_img + n_cols_img - 1) // n_cols_img
        fig_img, axes_img = plt.subplots(n_rows_img, n_cols_img, figsize=(n_cols_img * 5, n_rows_img * 4.5), squeeze=False) # Reduced size
        fig_img.suptitle("Final Embeddings with Face Images (Optimal Parameters)", fontsize=14)
        axes_flat_img = axes_img.flatten()
        plot_idx_img = 0
        for name in sorted(valid_optimal_embeddings_img.keys()):
            embedding = valid_optimal_embeddings_img[name]
            ax = axes_flat_img[plot_idx_img]
            params_used = optimal_params.get(name, {})
            ax.set_title(f"{name}\n{params_used}", fontsize=8)
            ax.set_xticks([]); ax.set_yticks([])
            x_coords = embedding[:, 0]; y_coords = embedding[:, 1]
            for i in range(face_data.shape[0]):
                x, y = x_coords[i], y_coords[i]
                img = face_data[i, :, :].T
                try:
                    imagebox = OffsetImage(img, zoom=0.25, cmap='gray')
                    imagebox.image.axes = ax
                    ab = AnnotationBbox(imagebox, (x, y), xycoords='data', frameon=False, pad=0.0)
                    ax.add_artist(ab)
                except Exception as e: print(f"Warn: Img {i} for {name}: {e}")
            ax.update_datalim(np.column_stack([x_coords, y_coords])); ax.autoscale_view()
            plot_idx_img += 1
        for j in range(plot_idx_img, len(axes_flat_img)): axes_flat_img[j].axis('off')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig_filename = os.path.join(output_dir_abs, "final_images.png") # Use absolute path
        plt.savefig(fig_filename)
        print(f"Saved: {fig_filename}")
        plt.close(fig_img)
    else: print("No valid optimal embeddings for image plotting.")
else: print("Skipping image plot: optimal_embeddings_dict or face_data missing.")


# Cell 7: Compute Parameter Tuning Results (Needed for Figures 4 & 5)
print("\n--- Computing Parameter Tuning Results ---")
k_tuning_results_list = []
tsne_tuning_results_list = []
param_tuning_results_k_df = pd.DataFrame()
param_tuning_results_tsne_df = pd.DataFrame()

if X_scaled is not None and ground_truth_order_indices is not None and compute_all_metrics is not None:
    print("Running K-Tuning loop...")
    neighbor_methods_funcs = {'Isomap': run_isomap, 'LLE': run_lle, 'LTSA': run_lle, 'MLLE': run_lle, 'HLLE': run_lle, 'Spectral': run_spectral}
    start_k_tune_time = time.time()
    for k in k_values_refined:
        current_k_embeddings = {}
        for name, func in neighbor_methods_funcs.items():
            if func is None: continue
            params = {'n_components': N_COMPONENTS, 'n_neighbors': k}
            if name not in ['Isomap']: params['random_state'] = RANDOM_STATE
            if name in ['LLE', 'LTSA', 'MLLE', 'HLLE']:
                if name == 'LLE': params['method'] = 'standard'
                elif name == 'LTSA': params['method'] = 'ltsa'
                elif name == 'MLLE': params['method'] = 'modified'
                elif name == 'HLLE': params['method'] = 'hessian'
                if name == 'HLLE' and k <= 5: continue # HLLE k requirement check
            try: embedding, _ = func(X_scaled, **params); current_k_embeddings[name] = embedding
            except Exception as e: print(f" K-Tune {name} (k={k}) failed: {e}"); current_k_embeddings[name] = None
        valid_k_embeddings = {n: e for n, e in current_k_embeddings.items() if e is not None}
        if valid_k_embeddings:
              metrics_df_k = compute_all_metrics(flattened_face_data, valid_k_embeddings, ground_truth_order_indices, n_neighbors=METRICS_K_FOR_TC)
              metrics_df_k['k'] = k
              k_tuning_results_list.extend(metrics_df_k.to_dict('records'))
    param_tuning_results_k_df = pd.DataFrame(k_tuning_results_list)
    print(f"K-Tuning loop completed in {time.time() - start_k_tune_time:.2f}s.")

    print("Running Perplexity-Tuning loop...")
    start_p_tune_time = time.time()
    if run_tsne:
        for p in perplexity_values_refined:
             if p >= X_scaled.shape[0] -1 : continue
             try:
                 tsne_embedding_p, _ = run_tsne(X_scaled, n_components=N_COMPONENTS, perplexity=p, random_state=RANDOM_STATE)
                 metrics_df_p = compute_all_metrics(flattened_face_data, {'t-SNE': tsne_embedding_p}, ground_truth_order_indices, n_neighbors=METRICS_K_FOR_TC)
                 if not metrics_df_p.empty: metrics_df_p['perplexity'] = p; tsne_tuning_results_list.extend(metrics_df_p.to_dict('records'))
             except Exception as e: print(f" P-Tune t-SNE (p={p}) failed: {e}")
        param_tuning_results_tsne_df = pd.DataFrame(tsne_tuning_results_list)
        print(f"Perplexity-Tuning loop completed in {time.time() - start_p_tune_time:.2f}s.")
    else: print("Skipping P-Tuning: run_tsne not available.")
else: print("Skipping Tuning Computations: Missing data or functions.")
print("--- Parameter Tuning Results Computed (or skipped) ---")


# Cell 8: Generate K-Tuning Plot (Figure 4 in report)
print("\n--- Generating Metrics vs K Plot ---")
if not param_tuning_results_k_df.empty:
    metrics_to_plot = ['TAE (min)', 'Trustworthiness', 'Continuity']
    n_metrics = len(metrics_to_plot)
    fig_k, axes_k = plt.subplots(n_metrics, 1, figsize=(10, n_metrics * 3.5), sharex=True)
    fig_k.suptitle('Metrics vs. Number of Neighbors (k)')
    for i, metric in enumerate(metrics_to_plot):
        ax = axes_k[i]
        sns.lineplot(data=param_tuning_results_k_df, x='k', y=metric, hue='Method', style='Method', markers=True, dashes=False, ax=ax, legend=(i == n_metrics - 1))
        ax.set_ylabel(metric); ax.grid(True, linestyle='--', alpha=0.7)
        if 'k_values_refined' in locals(): ax.set_xticks(k_values_refined)
        if i < n_metrics - 1: ax.set_xlabel('')
        else: ax.set_xlabel('Number of Neighbors (k)'); ax.legend(title='Method', bbox_to_anchor=(1.03, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout(rect=[0, 0, 0.88, 0.95])
    fig_filename = os.path.join(output_dir_abs, "tuning_k.png") # Use absolute path
    plt.savefig(fig_filename)
    print(f"Saved: {fig_filename}")
    plt.close(fig_k)
else: print("Skipping K-Tuning plot: Results not available.")


# Cell 9: Generate Perplexity-Tuning Plot (Figure 5 in report)
print("\n--- Generating Metrics vs Perplexity Plot ---")
if not param_tuning_results_tsne_df.empty:
    metrics_to_plot = ['TAE (min)', 'Trustworthiness', 'Continuity']
    n_metrics = len(metrics_to_plot)
    fig_p, axes_p = plt.subplots(n_metrics, 1, figsize=(8, n_metrics * 3), sharex=True)
    fig_p.suptitle('t-SNE Metrics vs. Perplexity')
    for i, metric in enumerate(metrics_to_plot):
        ax = axes_p[i]
        sns.lineplot(data=param_tuning_results_tsne_df, x='perplexity', y=metric, marker='o', ax=ax, errorbar=None)
        ax.set_ylabel(metric); ax.grid(True, linestyle='--', alpha=0.7)
        if 'perplexity_values_refined' in locals(): ax.set_xticks(perplexity_values_refined)
        if i < n_metrics - 1: ax.set_xlabel('')
        else: ax.set_xlabel('Perplexity')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fig_filename = os.path.join(output_dir_abs, "tuning_p.png") # Use absolute path
    plt.savefig(fig_filename)
    print(f"Saved: {fig_filename}")
    plt.close(fig_p)
else: print("Skipping Perplexity-Tuning plot: Results not available.")

print("\n--- Figure Export Script Complete ---")

--- Running Setup ---
Added '/Users/dingding/Desktop/大学博士阶段科研资料/课程资料/25spring/Topological and Geometric Data Reduction and Visualization/Project' to sys.path
Install it via: pip install scikit-learn-extra
Successfully imported functions from src.
Output directory exists: /Users/dingding/Desktop/大学博士阶段科研资料/课程资料/25spring/Topological and Geometric Data Reduction and Visualization/Project/figs
--- Setup Complete ---

--- Loading and Preparing Data ---
Keys in face.mat: dict_keys(['__header__', '__version__', '__globals__', 'Y', 'id'])
Data loaded successfully! Shape: (33, 92, 112)
Data reshaped successfully to: (33, 10304)
Ground Truth order defined.
Data Scaled using StandardScaler.
--- Data Prep Complete ---

--- Generating Sample Faces Plot ---
Saved: /Users/dingding/Desktop/大学博士阶段科研资料/课程资料/25spring/Topological and Geometric Data Reduction and Visualization/Project/figs/sample_faces.png

--- Computing Optimal Embeddings ---
Running PCA with optimal params...
Running PCA with n_component

/opt/anaconda3/envs/tgdr39/lib/python3.9/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


breaking at iteration 112 with stress 797016.0476177125
breaking at iteration 242 with stress 743479.9974281687
breaking at iteration 247 with stress 711784.9386160637
MDS completed.
Running Isomap with optimal params...
Running Isomap with n_components=2, n_neighbors=5...
Isomap completed.
Running LLE with optimal params...
Running LLE (method='standard') with n_components=2, n_neighbors=5...
LLE (method='standard') completed.
Running LTSA with optimal params...
Running LLE (method='ltsa') with n_components=2, n_neighbors=10...
LLE (method='ltsa') completed.
Running MLLE with optimal params...
Running LLE (method='modified') with n_components=2, n_neighbors=12...
LLE (method='modified') completed.
Running HLLE with optimal params...
Running LLE (method='hessian') with n_components=2, n_neighbors=10...
LLE (method='hessian') completed.
Running Spectral with optimal params...
Running Spectral Embedding with n_components=2, n_neighbors=5...
Spectral Embedding completed.
Running t-SNE wit

/opt/anaconda3/envs/tgdr39/lib/python3.9/site-packages/sklearn/manifold/_locally_linear.py:475: RuntimeWarning: invalid value encountered in scalar divide
  alpha_i = np.linalg.norm(Vi.sum(0)) / np.sqrt(s_i)
/opt/anaconda3/envs/tgdr39/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


LLE (method='ltsa') completed.
Running LLE (method='modified') with n_components=2, n_neighbors=7...
LLE (method='modified') completed.
Running LLE (method='hessian') with n_components=2, n_neighbors=7...
LLE (method='hessian') completed.
Running Spectral Embedding with n_components=2, n_neighbors=7...
Spectral Embedding completed.
Note: TAE for LTSA was calculated using reversed order (min(544, 18)).
Note: TAE for HLLE was calculated using reversed order (min(544, 18)).
Running Isomap with n_components=2, n_neighbors=8...
Isomap completed.
Running LLE (method='standard') with n_components=2, n_neighbors=8...
LLE (method='standard') completed.
Running LLE (method='ltsa') with n_components=2, n_neighbors=8...
LLE (method='ltsa') completed.
Running LLE (method='modified') with n_components=2, n_neighbors=8...
LLE (method='modified') completed.
Running LLE (method='hessian') with n_components=2, n_neighbors=8...
LLE (method='hessian') completed.
Running Spectral Embedding with n_component